Same as Really Stupid Summarization but it does Text Rank instead of avering the tf-idfs values.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

np.random.seed(1)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from nltk.tokenize import sent_tokenize

In [2]:
df = pd.read_csv("../data/bbc_text_cls.csv")

In [3]:
df.head()
article_to_summarize = df[df.labels == 'business']['text'].sample(random_state=42)

In [4]:
import textwrap

def wrap(x):
    return textwrap.fill(x, replace_whitespace=False, fix_sentence_endings=True)

In [5]:
print(wrap(article_to_summarize.iloc[0]))

Christmas sales worst since 1981

UK retail sales fell in December,
failing to meet expectations and making it by some counts the worst
Christmas since 1981.

Retail sales dropped by 1% on the month in
December, after a 0.6% rise in November, the Office for National
Statistics (ONS) said.  The ONS revised the annual 2004 rate of growth
down from the 5.9% estimated in November to 3.2%. A number of
retailers have already reported poor figures for December.  Clothing
retailers and non-specialist stores were the worst hit with only
internet retailers showing any significant growth, according to the
ONS.

The last time retailers endured a tougher Christmas was 23 years
previously, when sales plunged 1.7%.

The ONS echoed an earlier
caution from Bank of England governor Mervyn King not to read too much
into the poor December figures.  Some analysts put a positive gloss on
the figures, pointing out that the non-seasonally-adjusted figures
showed a performance comparable with 2003. The Novembe

In [6]:
# split on sentences
sentences = sent_tokenize(article_to_summarize.iloc[0].split("\n", 1)[1])
len(sentences)

17

In [7]:
# This is the only different part
def score_sentence(tfIdf_value):
    # Perform Text Rank
    
    # Steps

In [8]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/alindumitru/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alindumitru/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:

from nltk.corpus import stopwords
vectorizer = TfidfVectorizer(stop_words=stopwords.words('english'), norm = 'l1')

N = len(sentences)

TextRank_M = np.zeros(N, N)
from sklearn.metrics.pairwise import cosine_similarity

for i in range(N) - 1:
    for j in range(i, N):
        TextRank_M(i, j) = cosine_similarity(sentences(i), sentences(j))
        
# normalize
sums = TextRank_M.sum(axis=1, keepdims=True)
TextRank_M = TextRank_M / sums

alpha = 0.15

SmoothMatrix = np.full((N, N), 1/N)
TextRank_A = alpha * SmoothMatrix + (1 - alpha) * TextRank_M

tfIdfSentences_scores = []
for sentence in sentences:
    tfIdf_value = vectorizer.fit_transform([sentence])
    tfIdfSentences_scores.append(score_sentence(tfIdf_value))

In [10]:
sorted_indices = np.argsort(tfIdfSentences_scores).astype(int)[::-1]
sorted_sentences = [sentences[i] for i in sorted_indices]

In [11]:
nr_sentences = len(sorted_sentences)
# take 20% of the sentences or minimum of 4
first_20_porc_sentences = nr_sentences * 0.2
chosen_sent_index = first_20_porc_sentences if first_20_porc_sentences >= 4 else 4

summary = sorted_sentences[:chosen_sent_index]
for sentence in summary:
    print(sentence)

A number of retailers have already reported poor figures for December.
The ONS revised the annual 2004 rate of growth down from the 5.9% estimated in November to 3.2%.
However, reports from some High Street retailers highlight the weakness of the sector.
"Our view is the Bank of England will keep its powder dry and wait to see the big picture."
